# テストの実行

In [113]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

from tqdm import tqdm

In [114]:
pd.set_option('display.max_columns', None)

In [115]:
TESTSET_DIR = os.path.join('/home', 'jovyan', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jovyan', 'work')

PROG_PATH = os.path.join(PRJ_DIR, '10_tsplib_euc_2d', 'main')

## マスタの読み込み

In [116]:
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, 'tsplib', 'euc_2d.csv',))
sys_seed_df

,name,comment,dim,dist_lb,dist_ub
0,a280,drilling problem (Ludwig),280,2579,2579
1,berlin52,52 locations in Berlin (Groetschel),52,7542,7542
2,bier127,127 Biergaerten in Augsburg (Juenger/Reinelt),127,118282,118282
3,brd14051,BR Deutschland in den Grenzen von 1989 (Bachem...,14051,469385,469385
4,ch130,130 city problem (Churritz),130,6110,6110
...,...,...,...,...,...
72,u574,Drilling problem (Reinelt),574,36905,36905
73,u724,Drilling problem (Reinelt),724,41910,41910
74,usa13509,The file US.lat-long.Z can be found in the dir...,13509,19982859,19982859
75,vm1084,1084-city problem (Reinelt),1084,239297,239297


## 実行するロジックの指定

In [117]:
# 実行プログラムにタグをつけておく
## Champion管理用に単語は -(ハイフン) で区切る
PROG_TAG = 'first-prog'

In [118]:
def solve(seed):
    problem_path = os.path.join(TESTSET_DIR, 'tsplib', '01_euc_2d', '{}.in'.format(seed)) 
    command_str = 'echo {} | {}'.format(problem_path, PROG_PATH)
    # stack overflow対策
    # command_str = 'ulimit -S -s 1048576 && echo {} | {}'.format(problem_path, PROG_PATH)

    start_time = time.perf_counter()
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    
    #print('{}'.format(seed))    
    return (seed, e_time, res)

In [119]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_df = sys_seed_df

    # 1並列実行(正確に時間を測定するため)
    with futures.ThreadPoolExecutor(max_workers=1) as executor:
        seed_list = testset_df['name'].to_list()
        future_list = list(tqdm(executor.map(solve, seed_list), total=len(seed_list)))

    for future in future_list:
        seed, e_time, res = future
        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # -- start -- 生成コード貼り付け先
            elem_cnt = 3

            tour = str(res.stderr.decode('utf-8').split()[-elem_cnt + 0].replace('tour=', ''))
            distance = int(res.stderr.decode('utf-8').split()[-elem_cnt + 1].replace('distance=', ''))
            e_time_micro_sec = int(res.stderr.decode('utf-8').split()[-elem_cnt + 2].replace('e_time_micro_sec=', ''))

            solve_result.append(tour)
            solve_result.append(distance)
            solve_result.append(e_time_micro_sec)
            # -- end -- 生成コード貼り付け先

            # 相対スコア
            # rel_score = int(10 ** 9 * top_rate * champ_score_dict[seed] / score)
            # solve_result.append(rel_score)

        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    cols = ['testset', 'seed', 'time', 'tour', 'distance', 'e_time_micro_sec']
    result_df.columns = cols

    return result_df

In [120]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [121]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            # -- start -- 生成コード貼り付け先
            'distance_mean': [np.mean(test_result_df['distance'])],
            'e_time_micro_sec_mean': [np.mean(test_result_df['e_time_micro_sec'])],
            # -- end -- 生成コード貼り付け先

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    
    cols = ['tag']
    cols.extend(summary_df.columns)
    
    summary_all_df = summary_all_df[cols]
    
    return summary_all_df

In [122]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

#testset_name = '01_testset_pre'
testset_name = '02_testset_sys'
#testset_name = '03_testset_stress'
#testset_name = '04_testset_param'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2024-07-19 16:44:55,825: Start
100%|███████████████████████████████████████████████████████████████████████████████████| 77/77 [00:02<00:00, 31.99it/s]
2024-07-19 16:44:58,257: finish!


In [123]:
result_merge_df = pd.merge(result_df, sys_seed_df[['name', 'dim', 'dist_lb']], left_on='seed', right_on='name')
result_merge_df['approx_rate'] = result_merge_df['distance'] / result_merge_df['dist_lb']
result_merge_df.sort_values('e_time_micro_sec')

,testset,seed,time,tour,distance,e_time_micro_sec,name,dim,dist_lb,approx_rate
1,02_testset_sys,berlin52,1,1-22-49-32-36-35-34-39-40-38-37-48-24-5-15-6-4...,8980,58,berlin52,52,7542,1.190666
15,02_testset_sys,eil51,1,1-32-11-38-5-49-9-50-16-2-29-21-34-30-10-39-33...,511,70,eil51,51,426,1.199531
63,02_testset_sys,st70,1,1-36-23-47-16-37-58-50-10-5-53-66-22-63-59-38-...,830,89,st70,70,675,1.22963
16,02_testset_sys,eil76,1,1-73-33-63-16-3-44-32-9-39-72-58-10-38-65-11-6...,642,92,eil76,76,538,1.193309
50,02_testset_sys,pr76,1,1-2-23-22-21-25-24-46-45-44-48-47-69-68-67-50-...,153462,100,pr76,76,108159,1.418856
...,...,...,...,...,...,...,...,...,...,...
57,02_testset_sys,rl11849,257,1-11410-8533-9473-8296-8490-11748-1203-10732-9...,1125249,251920,rl11849,11849,923288,1.218741
74,02_testset_sys,usa13509,329,1-2-3-4-5-6-7-8-9-11-12-10-13-14-15-18-17-19-2...,24973197,322858,usa13509,13509,19982859,1.249731
3,02_testset_sys,brd14051,356,1-6-12-15-22-32-70-115-111-141-164-217-195-216...,585382,349861,brd14051,14051,469385,1.247125
7,02_testset_sys,d15112,410,1-13732-3004-9995-12789-1422-12416-10094-13422...,1960503,403029,d15112,15112,1573084,1.24628


In [125]:
np.mean(result_merge_df['approx_rate'])

1.2427925455269027

In [126]:
result_merge_df.to_csv('result.csv')